In [ ]:
import os
print("Setting up GCS access...")
from google.colab import auth
os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user()

# Set Up

In [2]:
from IPython.display import clear_output 
!pip install gcsfs
!pip install t5==0.9.2
!pip install -q tensorflow-text==2.8.0rc0
clear_output()

In [ ]:
print("Installing dependencies...")

import functools
import os
import gin
from contextlib import contextmanager
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import logging as py_logging
tf.app.flags.DEFINE_string ('f', '', '')

import t5

In [ ]:
# Set credentials for GCS reading/writing from Colab and TPU.
import tensorflow_gcs_config

TPU_TOPOLOGY = "2x2"
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  TPU_ADDRESS = tpu.get_master()
  print('Running on TPU:', TPU_ADDRESS)
except ValueError:
  raise BaseException(
    'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
tf.config.experimental_connect_to_host(TPU_ADDRESS)
tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

#LOGGING
tf.get_logger().propagate = False
py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

# Variables

In [ ]:
task = "masking" #@param ['masking']
scheduler = "noam" #@param ['noam']
VOCAB_PREFIX = 'tokenizer' #@param {type: "string"}
TOKENIZER_DIR = f"gs://lance2/tokenizer/"
path_pretraining_task1 = f"gs://lance2/pretraining/{task}/pretraining.tsv"
# Storage paths 
PRETRAIN_MODEL_DIR = f"gs://lance2/pretrained-model/{task}"



# Vocab

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


vocab_model_path = os.path.join(TOKENIZER_DIR, f'{VOCAB_PREFIX}.model')
vocab_path = os.path.join(TOKENIZER_DIR, f'{VOCAB_PREFIX}.vocab')


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask

num_special_mask_tokens = 100 #@param {type: "integer"}

def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, num_special_mask_tokens)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

In [ ]:
#Skip this cell for running the pre-training on the second task only
nq_tsv_path = {
    "train":      path_pretraining_task1,
}

num_nq_examples_task1 = dict(train=12671475)

def load_dataset(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

## Examples

In [ ]:
print("A few raw train examples...")
for ex in tfds.as_numpy(load_dataset("train").take(5)):
  print(ex)

# Dataset Preprocessing

In [ ]:
def preprocessing(ds):

  def to_inputs_and_targets(ex):
        inputs = tf.strings.join(['MASKING: ' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Examples

In [ ]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing(load_dataset("train").take(5)))
for ex in sample:
  print(ex)

# Set up task

In [ ]:
# Modify task registry     
t5.data.TaskRegistry.remove('masking')
t5.data.TaskRegistry.add(
    "masking",
    dataset_fn=load_dataset,
    splits=["train"],
    text_preprocessor=preprocessing,
    output_features = DEFAULT_OUTPUT_FEATURES,
    num_input_examples=num_nq_examples_task1
)

## Mixture

In [ ]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("pretraining")
t5.data.MixtureRegistry.add(
    "pretraining",
    ["masking"],
    default_rate=_rate_num_input_examples
)

## Model

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import learning_rate_schedule_noam
from t5 import models

# Model properties
MODEL_SIZE = "small" 
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 128, 16), #change batch 128
    "base": (2, 16, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(PRETRAIN_MODEL_DIR)

# Mesh Tensorflow Transformer
model = models.mtf_model.MtfModel(
    model_dir=PRETRAIN_MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = learning_rate_schedule_noam,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=10000,
    keep_checkpoint_max=keep_checkpoint_max,
    iterations_per_loop=100,
)

# Set up learning Scheduler

In [ ]:
remote_gin_path = f"gs://lance2/learning_rate_scheduler/{scheduler}/operative_config.gin"
local_gin_path = "/content/operative_config.gin"
!gsutil -m cp $remote_gin_path $local_gin_path

In [ ]:
with gin.unlock_config():
    gin.parse_config_file(local_gin_path)

# Pre-train model

In [ ]:
TRAIN_STEPS = 500000 #@param {type: "integer"}
model.train("pretraining", TRAIN_STEPS) #pretraining = mixture name 
